**LAMA :  Language Model Analysis**

# Downloading Audio File from YT link

In [ ]:
# Install necessary Python packages
!pip install yt-dlp pydub pytube opencv-python transformers huggingface_hub moviepy langdetect openai-whisper

# Install system package ffmpeg
!apt-get update && apt-get install -y ffmpeg
!pip install pymongo
!apt-get install -y imagemagick

# Upgrade moviepy
!pip install --upgrade moviepy

# Install additional Python packages
!pip install nltk
!pip install playwright
!playwright install
!pip install pandas
!pip install torch


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.1/170.1 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 30.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 28.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.1/168.1 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.3/157.3 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
with open('/etc/ImageMagick-6/policy.xml', 'w') as f:
    f.write('''
    <?xml version="1.0" encoding="UTF-8"?>
    <policymap>
      <policy domain="path" rights="read|write" pattern="@*" />
      <policy domain="path" rights="read|write" pattern="*" />
      <policy domain="system" rights="read|write" pattern="*" />
      <policy domain="resource" name="memory" value="2GiB" />
      <policy domain="resource" name="map" value="2GiB" />
      <policy domain="resource" name="disk" value="5GiB" />
      <policy domain="resource" name="file" value="768" />
      <policy domain="resource" name="thread" value="8" />
    </policymap>
    ''')


In [ ]:
import yt_dlp
import json
import whisper
from io import BytesIO
import numpy as np
from moviepy.editor import VideoFileClip, TextClip, CompositeVideoClip, concatenate_videoclips
from IPython.display import Video, display
from pymongo import MongoClient
from pydub import AudioSegment
import os
import tempfile
import nltk
from collections import Counter
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
import nest_asyncio
from playwright.async_api import async_playwright
import asyncio
import pandas as pd
from transformers import pipeline
import csv


# Creating Highlighted Videos


Download video from youtube

In [ ]:
def download_video_yt_dlp(url, output_file_name):
    ydl_opts = {
        'format': 'best',
        'outtmpl': output_file_name,
    }
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])
        print("Download video complete!")
    except Exception as e:
        print(f"An error occurred: {e}")

Funtion for formatting the transcripted text

In [ ]:
def format_with_timestamps(result, file_name):
    formatted_segments = []
    for segment in result.get("segments", []):
        segment_data = {
            "id": segment.get("id"),
            "start": round(segment.get("start", 0), 2),
            "end": round(segment.get("end", 0), 2),
            "text": segment.get("text", "")
        }
        formatted_segments.append(segment_data)

    # Save to a JSON file
    with open(file_name, 'w') as json_file:
        json.dump(formatted_segments, json_file, indent=4)
    print("Formatted subtitles created and saved")

Transcripted from audio to text by this function

In [ ]:

def process_video_to_audio(video_file_name, subtitle_file):
    # Remove the extension from the video file name
    base_name = os.path.splitext(video_file_name)[0]

    # Construct paths
    video_path = f'/content/{video_file_name}'
    audio_file = f'/content/{base_name}_audio.wav'

    # Extract audio from the video file using pydub
    audio = AudioSegment.from_file(video_path, format="mp4")

    # Save the audio to a WAV file (recommended for Whisper)
    audio.export(audio_file, format="wav")

    # Load the Whisper model
    model = whisper.load_model("base")

    # Transcribe the audio using Whisper
    result = model.transcribe(audio_file)

    print("Subtitle created by Whisper")
    print(result)

    # Format and save the transcription with timestamps
    format_with_timestamps(result, subtitle_file)



Search the keyword from the transcripted text

In [ ]:


def search_words_in_transcription(formatted_segments_file, search_words):
    search_words = [word.lower() for word in search_words]
    segments_with_words = []

    try:
        with open(formatted_segments_file, 'r') as file:
            formatted_segments = json.load(file)
    except FileNotFoundError:
        raise FileNotFoundError(f"The file {formatted_segments_file} was not found.")
    except json.JSONDecodeError:
        raise ValueError(f"Error decoding JSON from the file {formatted_segments_file}.")

    if not isinstance(formatted_segments, list):
        raise ValueError("Expected formatted_segments to be a list of dictionaries.")
    if not all(isinstance(seg, dict) for seg in formatted_segments):
        raise ValueError("Each item in formatted_segments should be a dictionary.")

    for segment in formatted_segments:
        if "text" in segment:
            text = segment["text"].lower()
            if any(word in text for word in search_words):
                id = segment.get("id", len(segments_with_words) + 1)
                start = segment.get("start")
                end = segment.get("end")
                segments_with_words.append({
                    "id": id,
                    "start_time": start,  # Renaming "start" to "start_time"
                    "end_time": end,      # Renaming "end" to "end_time"
                    "text": segment["text"]
                })

    # Check if segments_with_words is empty before saving
    if not segments_with_words:
        print("No matching words were found in the transcription. No file will be created.")
        return False

    return segments_with_words


Function for Creating the video based on keyword

In [ ]:
def create_highlight_video(original_video_path, highlighted_sections):
    clips = []
    original_video_path = '/content/' + original_video_path

    # Load the original video
    video = VideoFileClip(original_video_path)

    for segment in highlighted_sections:
        start = max(segment["start_time"], 0)
        end = min(segment["end_time"], video.duration)

        # Extract the clip
        clip = video.subclip(start, end)

        # Ensure clip is valid
        #print(f"Processing segment: {start}-{end} (Duration: {clip.duration})")

        # Create a TextClip for the corresponding text (subtitle) with white bold text and lighter black background
        text = segment["text"]
        text_clip = TextClip(
            text, fontsize=14, font='Arial-Bold', color='white', bg_color='#2c2c2c',  # Lighter dark gray
            method='caption', size=(clip.w, None)
        ).set_duration(clip.duration).set_position(("center", "bottom")).set_opacity(0.6)  # Reduced opacity for transparency

        # Overlay the subtitle onto the video clip
        video_with_subtitle = CompositeVideoClip([clip, text_clip])

        # Add the video with subtitles to the clips list
        clips.append(video_with_subtitle)

    # Concatenate all clips into a single video
    if clips:
        final_video = concatenate_videoclips(clips, method="compose")

        # Create a temporary file to save the video
        with tempfile.NamedTemporaryFile(delete=False, suffix=".mp4") as temp_file:
            temp_file_name = temp_file.name
            final_video.write_videofile(temp_file_name, codec="libx264", audio_codec="aac", fps=30)

        # Display the video
        print("Displaying the highlight video:")
        display(Video(temp_file_name, width=640, height=480, embed=True))

        # Clean up the temporary file
        os.remove(temp_file_name)
    else:
        print("No valid segments found to create the highlight video.")

Connection with mongodb database

In [ ]:

# Replace with your MongoDB connection string
connection_string = "mongodb+srv:cluster0.wg4rh.mongodb.net/?retryWrites=true&w=majority&appName=Cluster0"
# Initialize MongoDB client and access database
client = MongoClient(connection_string)
db = client['youtube_video_details']
videos_collection = db['video_information']

Deleting the temporary file

In [ ]:
def delete_all_files_in_directory(directory='/content/'):
    # Check if the directory exists
    if os.path.exists(directory):
        # Loop through all files and subdirectories
        for filename in os.listdir(directory):
            file_path = os.path.join(directory, filename)

            # Check if it's a file or directory
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)  # Delete the file
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)  # Delete the directory

        print(f"All files and directories in {directory} have been deleted.")
    else:
        print(f"The directory {directory} does not exist.")

Extract the highlighted section

In [ ]:
def extract_highlighted_sections(highlighted_subtitle_file):
    highlighted_sections = []
    for entry in highlighted_subtitle_file:
        highlighted_sections.append({
            "start_time": entry.get("start_time"),
            "end_time": entry.get("end_time"),
            "text": entry.get("text")
        })
    return highlighted_sections


Main function for creating the highlighted video from the given user keywords and youtube video url , finally store data in Mongodb database

In [ ]:
def create_video_json():
    video_url = input("Enter the video URL: ")
    keywords = [keyword.strip() for keyword in input("Enter keywords (comma-separated): ").split(',')]

    existing_video = videos_collection.find_one({"video_url": video_url})

    if not existing_video:
        unique_id = videos_collection.count_documents({}) + 1
        subtitle_file = f"subtitle.json"
        video_file_name = f"video.mp4"

        # Download and process the video
        download_video_yt_dlp(video_url, video_file_name)
        process_video_to_audio(video_file_name, subtitle_file)

        # Search for keywords in transcription
        #highlighted_subtitle_files = f"highlighted_video_{unique_id}_1.json"
        highlighted_subtitle_files = search_words_in_transcription(subtitle_file, keywords)

        if not highlighted_subtitle_files:
            return

        # Extract highlighted sections from the subtitle file
        highlighted_sections = extract_highlighted_sections(highlighted_subtitle_files)

        # Create highlight video using the extracted sections
        create_highlight_video(video_file_name, highlighted_sections)

        # Create the new video entry in the required format
        new_video = {
            "video_url": video_url,
            "details": [
                {
                    "keywords": [
                        {
                            "keyword": keywords,
                            "highlighted_sections": highlighted_sections
                        }
                    ]
                }
            ]
        }

        # Insert the new video document into the collection
        videos_collection.insert_one(new_video)
        print(f"New entry added with unique ID: {unique_id}")


    else:
        unique_id = existing_video['_id']
        video_file_name = f"video.mp4"
        subtitle_file = f"subtitle.json"
        existing_details = existing_video['details'][0]

        updated = False

        # Check if the keywords already exist in the video entry
        for keyword_entry in existing_details['keywords']:
            if set(keyword_entry['keyword']) == set(keywords):
                print("Keywords match existing entry.")
                # Download and process the video
                download_video_yt_dlp(video_url, video_file_name)
                highlighted_sections = keyword_entry['highlighted_sections']
                create_highlight_video(video_file_name, highlighted_sections)
                return

        # Download and process the video
        download_video_yt_dlp(video_url, video_file_name)
        process_video_to_audio(video_file_name, subtitle_file)

        # If the keywords don't match, process new highlighted sections
        #new_highlighted_files = f"highlighted_video_{unique_id}_{len(existing_details['keywords']) + 1}.json"
        new_highlighted_files = search_words_in_transcription(subtitle_file, keywords)

        print(new_highlighted_files)
        if not new_highlighted_files:
            return

        # Extract highlighted sections for the new keywords
        highlighted_sections = extract_highlighted_sections(new_highlighted_files)
        print(highlighted_sections)
        # Create highlight video using the new highlighted sections
        create_highlight_video(video_file_name, highlighted_sections)
        print(f"New entry added for keywords: {keywords} with highlighted files: {new_highlighted_files}")

        new_keywords = [{
            "keyword": keywords,  # Storing all keywords as a list
            "highlighted_sections": highlighted_sections
        }]

        # Update the document by appending the new keywords and highlighted sections
        videos_collection.update_one(
            {"_id": unique_id},
            {"$push": {"details.0.keywords": {"$each": new_keywords}}}
        )


Calling the main function and calculating the total time taken

In [ ]:
import time
start_time = time.time()
create_video_json()# Call for mongodb storage
end_time = time.time()  # Capture the end time
execution_time = end_time - start_time  # Calculate the total execution time
print(f"Total execution time: {execution_time:.2f} seconds")

Checking frame rate in the video

In [ ]:
from moviepy.editor import VideoFileClip

def get_frame_rate(video_path):
    video = VideoFileClip(video_path)
    return video.fps

# Path to the video file
video_path = '/content/video.mp4'

frame_rate = get_frame_rate(video_path)
print(f"Frame rate: {frame_rate} fps")
print(f"Number of frames in one second: {frame_rate}")


Frame rate: 29.97002997002997 fps
Number of frames in one second: 29.97002997002997


# Use XML and json file storage system instead of Mongodb for comparing

In [ ]:
# Call the function
# #delete_all_files_in_directory('/content/')

In [ ]:
import json
import xml.etree.ElementTree as ET
import os

# Utility function to save data to a JSON file
def save_to_json(filename, data):
    with open(filename, 'w') as f:
        json.dump(data, f, indent=4)

# Utility function to load all data from a JSON file
def load_all_from_json(filename):
    if os.path.exists(filename):
        with open(filename, 'r') as f:
            try:
                data = json.load(f)
                if isinstance(data, dict):  # Single video entry case
                    return [data]
                elif isinstance(data, list):  # List of video entries case
                    return data
            except json.JSONDecodeError as e:
                print(f"Error loading JSON file: {e}")
                return []
    else:
        print(f"{filename} file not found")
        return []

# Utility function to save data to an XML file
def save_to_xml(filename, data):
    root = ET.Element('Videos')

    for video_data in data:
        video_elem = ET.SubElement(root, 'Video')
        ET.SubElement(video_elem, 'URL').text = video_data['video_url']

        details_elem = ET.SubElement(video_elem, 'Details')
        for detail in video_data['details']:
            for keyword_data in detail['keywords']:
                keyword_elem = ET.SubElement(details_elem, 'Keyword')
                keyword_text = ','.join(keyword_data['keyword'])
                ET.SubElement(keyword_elem, 'Text').text = keyword_text
                sections_elem = ET.SubElement(keyword_elem, 'HighlightedSections')
                for section in keyword_data['highlighted_sections']:
                    section_elem = ET.SubElement(sections_elem, 'Section')
                    ET.SubElement(section_elem, 'StartTime').text = str(section['start_time'])
                    ET.SubElement(section_elem, 'EndTime').text = str(section['end_time'])
                    ET.SubElement(section_elem, 'Text').text = section['text']

    tree = ET.ElementTree(root)
    tree.write(filename)

def load_all_from_xml(filename, input_keywords, video_url):
    if not os.path.exists(filename):
        print(f"{filename} file not found")
        return []

    try:
        tree = ET.parse(filename)
        root = tree.getroot()

        data = []
        input_keywords_set = set([k.strip().lower() for k in input_keywords])  # Normalize input keywords

        # Loop through each Video element in the XML
        for video_elem in root.findall('Video'):
            # Check if the video URL matches the input video_url
            video_elem_url = video_elem.findtext('URL')
            if video_elem_url == video_url:
                video_data = {"video_url": video_elem_url, "details": []}

                # Loop through each Keyword in the Details section
                for keyword_elem in video_elem.findall('Details/Keyword'):
                    xml_keywords = keyword_elem.findtext('Text').split(',')  # Split XML keywords
                    xml_keywords_set = set([k.strip().lower() for k in xml_keywords])  # Normalize case and whitespace

                    # Check if the input keywords match exactly with the XML keywords set
                    if xml_keywords_set == input_keywords_set:
                        highlighted_sections = []
                        # Loop through each Section and extract highlighted sections
                        for section_elem in keyword_elem.findall('HighlightedSections/Section'):
                            highlighted_sections.append({
                                'start_time': float(section_elem.findtext('StartTime')),
                                'end_time': float(section_elem.findtext('EndTime')),
                                'text': section_elem.findtext('Text')
                            })
                        # Add the matching keyword and its highlighted sections to the details
                        video_data['details'].append({
                            "keywords": [{"keyword": xml_keywords, "highlighted_sections": highlighted_sections}]
                        })

                # If details were found, add the video data to the result list
                if video_data['details']:
                    data.append(video_data)

                # Exit the loop since the video URL has already matched
                break

        return data
    except ET.ParseError as e:
        print(f"Error loading XML file: {e}")
        return []

# Main function to handle video processing
def create_video_json_or_xml(storage_format='json'):
    video_url = input("Enter the video URL: ")
    keywords = [keyword.strip() for keyword in input("Enter keywords (comma-separated): ").split(',')]

    # Determine file type based on storage format
    video_data_file = f"videos.{storage_format}"

    # Load existing data based on the storage format
    if storage_format == 'json':
        existing_videos = load_all_from_json(video_data_file)
    else:
        existing_videos = load_all_from_xml(video_data_file,keywords,video_url)

    # Check if the video URL already exists in the loaded data
    existing_video_entry = next((v for v in existing_videos if v['video_url'] == video_url), None)
    print(existing_video_entry)
    if existing_video_entry:
        existing_details = existing_video_entry['details'][0]

        # Check if the keywords already exist in the video entry
        for keyword_entry in existing_details['keywords']:
            if set(keyword_entry['keyword']) == set(keywords):
                print("Keywords match existing entry.")
                # Create highlight video for the existing keywords
                video_file_name = f"video_{storage_format}.mp4"
                download_video_yt_dlp(video_url, video_file_name)
                create_highlight_video(video_file_name, keyword_entry['highlighted_sections'])
                return

        # If keywords don't match, process the new keywords
        subtitle_file = f"subtitle_{storage_format}.json"
        download_video_yt_dlp(video_url, f"video_{storage_format}.mp4")
        process_video_to_audio(f"video_{storage_format}.mp4", subtitle_file)
        new_highlighted_files = search_words_in_transcription(subtitle_file, keywords)

        if not new_highlighted_files:
            return

        highlighted_sections = extract_highlighted_sections(new_highlighted_files)
        create_highlight_video(f"video_{storage_format}.mp4", highlighted_sections)

        # Add new keywords and highlighted sections to existing video entry
        new_keywords = {
            "keyword": keywords,
            "highlighted_sections": highlighted_sections
        }
        existing_video_entry['details'][0]['keywords'].append(new_keywords)

        # Save the updated data to file
        if storage_format == 'json':
            save_to_json(video_data_file, existing_videos)
        else:
            save_to_xml(video_data_file, existing_videos)

        print(f"Updated entry added for keywords: {keywords} with new highlighted sections.")
        return

    # If the video URL is not found, process it as a new video
    subtitle_file = f"subtitle_{storage_format}.json"
    download_video_yt_dlp(video_url, f"video_{storage_format}.mp4")
    process_video_to_audio(f"video_{storage_format}.mp4", subtitle_file)
    highlighted_subtitle_files = search_words_in_transcription(subtitle_file, keywords)

    if not highlighted_subtitle_files:
        return

    highlighted_sections = extract_highlighted_sections(highlighted_subtitle_files)
    create_highlight_video(f"video_{storage_format}.mp4", highlighted_sections)

    # Create new video entry
    new_video = {
        "video_url": video_url,
        "details": [
            {
                "keywords": [
                    {
                        "keyword": keywords,
                        "highlighted_sections": highlighted_sections
                    }
                ]
            }
        ]
    }

    # Append new video entry to the existing data
    existing_videos.append(new_video)

    # Save the updated data to file
    if storage_format == 'json':
        save_to_json(video_data_file, existing_videos)
    else:
        save_to_xml(video_data_file, existing_videos)

    print(f"New entry added for video URL: {video_url}")


Calling the main funciton for json file storage system

In [ ]:
import time
start_time = time.time()
create_video_json_or_xml(storage_format='json')  # Call for JSON storage
end_time = time.time()  # Capture the end time
execution_time = end_time - start_time  # Calculate the total execution time
print(f"Total execution time: {execution_time:.2f} seconds")

Calling main funciton for the xml file storage system

In [ ]:
start_time = time.time()
create_video_json_or_xml(storage_format='xml')   # Call for XML storage
end_time = time.time()  # Capture the end time
execution_time = end_time - start_time  # Calculate the total execution time
print(f"Total execution time: {execution_time:.2f} seconds")

# Frequency of words finding

In [ ]:
# Download required NLTK data
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

# Load JSON data from file
with open('/content/subtitle.json', 'r') as file:
    data = json.load(file)

# Extract text from JSON
text = ' '.join(segment['text'] for segment in data)

# Tokenize and POS tagging
tokens = word_tokenize(text)
tagged_tokens = pos_tag(tokens)

# Define POS tags for each part of speech
POS_TAGS = {
    'NN': 'Noun', 'NNS': 'Noun', 'NNP': 'Noun', 'NNPS': 'Noun', # Nouns
    'JJ': 'Adjective', 'JJR': 'Adjective', 'JJS': 'Adjective',    # Adjectives
    'RB': 'Adverb', 'RBR': 'Adverb', 'RBS': 'Adverb',             # Adverbs
    'VB': 'Verb', 'VBD': 'Verb', 'VBG': 'Verb', 'VBN': 'Verb', 'VBP': 'Verb', 'VBZ': 'Verb'  # Verbs
}

# Define auxiliary verbs and specific adverbs to exclude
auxiliary_verbs = set([
    'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being',
    'have', 'has', 'had', 'having',
    'do', 'does', 'did', 'doing', "'s", "'m", "'re", "n't"
])

# Define adverbs to exclude
specific_adverbs_to_exclude = set([
    'so', "n't", 'just', 'right', 'not', 'here', 'maybe', 'then',  # Original adverbs
    'often', 'always', 'never', 'rarely',  # Frequency adverbs
    'very', 'too', 'quite',  # Intensity adverbs
    'everywhere', 'nowhere', 'somewhere', 'anywhere',  # Place adverbs
])

# Function to filter and count words by POS tag
def filter_words_by_pos(tagged_tokens, pos_tag_prefix):
    return [word.lower() for word, tag in tagged_tokens if tag.startswith(pos_tag_prefix)]

# Function to remove auxiliary verbs and specific adverbs
def remove_unwanted_words(words):
    return [word for word in words if word not in auxiliary_verbs and word not in specific_adverbs_to_exclude]

# Find and print the top 10 most frequent words for each POS
for pos_tag, pos_name in POS_TAGS.items():
    print(f"\nTop 10 {pos_name} frequencies (excluding unwanted words):")
    filtered_words = filter_words_by_pos(tagged_tokens, pos_tag)
    filtered_words = remove_unwanted_words(filtered_words)
    filtered_word_freq = Counter(filtered_words)
    most_common = filtered_word_freq.most_common(10)
    for word, freq in most_common:
        print(f"{word}: {freq}")

# Aggregate filtered words from all POS tags
all_filtered_words = []
for pos_tag, pos_name in POS_TAGS.items():
    filtered_words = filter_words_by_pos(tagged_tokens, pos_tag)
    filtered_words = remove_unwanted_words(filtered_words)
    all_filtered_words.extend(filtered_words)

# Count frequencies of all filtered words
all_word_freq = Counter(all_filtered_words)

# Find and print the top 10 most frequent words overall
most_common = all_word_freq.most_common(10)
#print("\nTop 10 most frequent words (excluding unwanted words):")
for word, freq in most_common:
    print(f"{word}: {freq}")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!



Top 10 Noun frequencies (excluding unwanted words):
task: 20
%: 13
words: 11
band: 11
overview: 9
time: 8
data: 8
charts: 7
school: 7
changes: 6

Top 10 Noun frequencies (excluding unwanted words):
words: 11
charts: 7
changes: 6
data: 5
maps: 4
appliances: 4
stages: 4
errors: 4
bottles: 4
answers: 3

Top 10 Noun frequencies (excluding unwanted words):
ielts: 5
band: 4
cambridge: 2
remember: 2
time: 1
wait: 1
well: 1
back: 1
first: 1
taster: 1

Top 10 Noun frequencies (excluding unwanted words):

Top 10 Adjective frequencies (excluding unwanted words):
first: 8
important: 6
third: 6
main: 5
good: 5
vocabulary: 5
possible: 4
second: 4
new: 4
key: 3

Top 10 Adjective frequencies (excluding unwanted words):
higher: 2
less: 2
better: 2
easier: 1
older: 1
lower: 1
more: 1

Top 10 Adjective frequencies (excluding unwanted words):
least: 1
best: 1
most: 1

Top 10 Adverb frequencies (excluding unwanted words):
also: 10
really: 5
together: 5
well: 4
only: 3
there: 2
notice: 2
again: 2
even: 2
n

In [ ]:
import nltk
from collections import Counter

# Download required NLTK data
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

# Load JSON data from file
with open('/content/subtitle_json.json', 'r') as file:
    data = json.load(file)

# Extract text from JSON
text = ' '.join(segment['text'] for segment in data)

# Tokenize and POS tagging
tokens = word_tokenize(text)
tagged_tokens = pos_tag(tokens)

# Define POS tags for nouns, adjectives, adverbs, and verbs
POS_TAGS = {
    'NN': 'Noun', 'NNS': 'Noun', 'NNP': 'Noun', 'NNPS': 'Noun',  # Nouns
    'JJ': 'Adjective', 'JJR': 'Adjective', 'JJS': 'Adjective',   # Adjectives
    'RB': 'Adverb', 'RBR': 'Adverb', 'RBS': 'Adverb',            # Adverbs
    'VB': 'Verb', 'VBD': 'Verb', 'VBG': 'Verb', 'VBN': 'Verb', 'VBP': 'Verb', 'VBZ': 'Verb'  # Verbs
}

# Define auxiliary verbs and specific adverbs to exclude
auxiliary_verbs = set([
    'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being',
    'have', 'has', 'had', 'having',
    'do', 'does', 'did', 'doing', "'s", "'m", "'re", "n't"
])

# Define adverbs to exclude
specific_adverbs_to_exclude = set([
    'so', "n't", 'just', 'right', 'not', 'here', 'maybe', 'then',  # Original adverbs
    'often', 'always', 'never', 'rarely',  # Frequency adverbs
    'very', 'too', 'quite',  # Intensity adverbs
    'everywhere', 'nowhere', 'somewhere', 'anywhere','%',  # Place adverbs
])

# Function to filter and count words by POS tag
def filter_words_by_pos(tagged_tokens, pos_tag_prefix):
    return [word.lower() for word, tag in tagged_tokens if tag.startswith(pos_tag_prefix)]

# Function to remove auxiliary verbs and specific adverbs
def remove_unwanted_words(words):
    return [word for word in words if word not in auxiliary_verbs and word not in specific_adverbs_to_exclude]

# Filter by nouns, adjectives, verbs, and adverbs
filtered_words = []
for pos_tag, pos_name in POS_TAGS.items():
    filtered_pos_words = filter_words_by_pos(tagged_tokens, pos_tag)
    filtered_pos_words = remove_unwanted_words(filtered_pos_words)
    filtered_words.extend(filtered_pos_words)

# Count frequencies of all filtered words
all_word_freq = Counter(filtered_words)

# Sort and print all words by frequency (no limit)
print("\nWord frequencies (excluding unwanted words):")
for word, freq in all_word_freq.most_common():
    print(f"{word}: {freq}")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!



Word frequencies (excluding unwanted words):
words: 22
task: 20
band: 15
charts: 14
data: 13
changes: 12
show: 12
first: 12
see: 12
ielts: 10
let: 10
also: 10
time: 9
overview: 9
stages: 9
maps: 8
appliances: 8
presents: 8
errors: 8
bottles: 8
recycling: 8
use: 8
linked: 8
score: 7
look: 7
school: 7
plastic: 7
spent: 7
know: 7
answers: 6
descriptors: 6
requirements: 6
sentence: 6
devices: 6
well: 6
vocabulary: 6
sentences: 6
homes: 6
future: 6
sports: 6
process: 6
crushing: 6
important: 6
third: 6
going: 6
write: 6
want: 6
possessed: 6
need: 6
shows: 6
describing: 6
example: 5
ownership: 5
point: 5
main: 5
good: 5
really: 5
together: 5
go: 5
learn: 5
examiners: 4
cambridge: 4
features: 4
report: 4
ways: 4
question: 4
answer: 4
examiner: 4
families: 4
growth: 4
tasks: 4
remember: 4
synonyms: 4
years: 4
quarters: 4
courses: 4
structures: 4
buildings: 4
steps: 4
pellets: 4
key: 4
higher: 4
possible: 4
less: 4
second: 4
new: 4
better: 4
selected: 4
provide: 4
devoted: 4
using: 4
call: 4
m

In [ ]:
import nltk
from collections import Counter

# Download required NLTK data
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

# Load JSON data from file
with open('/content/subtitle.json', 'r') as file:
    data = json.load(file)

# Extract text from JSON
text = ' '.join(segment['text'] for segment in data)

# Tokenize and POS tagging
tokens = word_tokenize(text)
tagged_tokens = pos_tag(tokens)

# Define POS tags for nouns only
NOUN_TAGS = {
    'NN': 'Noun', 'NNS': 'Noun', 'NNP': 'Proper Noun', 'NNPS': 'Proper Noun'  # Nouns and Proper Nouns
}

# Function to filter nouns
def filter_nouns(tagged_tokens):
    return [word.lower() for word, tag in tagged_tokens if tag in NOUN_TAGS]

# Extract and filter only nouns
filtered_nouns = filter_nouns(tagged_tokens)

# Count frequencies of all filtered nouns
noun_freq = Counter(filtered_nouns)

# Sort and print all nouns by frequency (no limit)
print("\nNoun frequencies (excluding unwanted words):")
for word, freq in noun_freq.most_common():
    print(f"{word}: {freq}")



Noun frequencies (excluding unwanted words):
task: 20
%: 13
words: 11
band: 11
overview: 9
time: 8
data: 8
charts: 7
school: 7
changes: 6
score: 6
sentence: 6
process: 6
ielts: 5
example: 5
ownership: 5
look: 5
plastic: 5
maps: 4
appliances: 4
report: 4
question: 4
answer: 4
examiner: 4
stages: 4
growth: 4
synonyms: 4
errors: 4
bottles: 4
today: 3
answers: 3
type: 3
information: 3
achievement: 3
descriptors: 3
requirements: 3
home: 3
devices: 3
point: 3
range: 3
sentences: 3
homes: 3
year: 3
peak: 3
fall: 3
lot: 3
future: 3
site: 3
sports: 3
field: 3
car: 3
park: 3
diagram: 3
diagrams: 2
criteria: 2
examiners: 2
cambridge: 2
amount: 2
country: 2
features: 2
ways: 2
idea: 2
part: 2
criterion: 2
word: 2
presents: 2
percentage: 2
families: 2
tasks: 2
remember: 2
resource: 2
so: 2
verb: 2
paragraph: 2
right: 2
context: 2
course: 2
years: 2
quarters: 2
change: 2
courses: 2
exam: 2
grammar: 2
structures: 2
entrance: 2
buildings: 2
road: 2
side: 2
something: 2
recycling: 2
steps: 2
stage: 2


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
from collections import Counter

# Assuming all_filtered_words is the list of filtered words
all_word_freq = Counter(all_filtered_words)

# Sort the word frequencies in descending order
sorted_word_freq = all_word_freq.most_common()

# Display the sorted frequencies
for word, freq in sorted_word_freq:
    print(f"{word}: {freq}")


words: 22
task: 20
band: 15
charts: 14
data: 13
%: 13
changes: 12
show: 12
first: 12
see: 12
ielts: 10
let: 10
also: 10
time: 9
overview: 9
stages: 9
maps: 8
appliances: 8
presents: 8
errors: 8
bottles: 8
recycling: 8
use: 8
linked: 8
score: 7
look: 7
school: 7
plastic: 7
spent: 7
know: 7
answers: 6
descriptors: 6
requirements: 6
sentence: 6
devices: 6
well: 6
vocabulary: 6
sentences: 6
homes: 6
future: 6
sports: 6
process: 6
crushing: 6
important: 6
third: 6
going: 6
write: 6
want: 6
possessed: 6
need: 6
shows: 6
describing: 6
example: 5
ownership: 5
point: 5
main: 5
good: 5
really: 5
together: 5
go: 5
learn: 5
examiners: 4
cambridge: 4
features: 4
report: 4
ways: 4
question: 4
answer: 4
examiner: 4
families: 4
growth: 4
tasks: 4
remember: 4
synonyms: 4
years: 4
quarters: 4
courses: 4
structures: 4
buildings: 4
steps: 4
pellets: 4
key: 4
higher: 4
possible: 4
less: 4
second: 4
new: 4
better: 4
selected: 4
provide: 4
devoted: 4
using: 4
call: 4
maintained: 4
collocate: 4
looking: 4
pla

In [ ]:
top_keywords=[]
i=1
for word, freq in most_common:
    if i>3:
      break
    i+=1
    top_keywords.append(word)
    print(f"{word}: {freq}")

words: 22
task: 20
band: 15


In [ ]:
highlighted_subtitle_files = search_words_in_transcription("subtitle_json.json", top_keywords)

# Extract highlighted sections from the subtitle file
highlighted_sections = extract_highlighted_sections(highlighted_subtitle_files)

# Create highlight video using the extracted sections
create_highlight_video("video.mp4", highlighted_sections)

# Comment Analysis and video create

In [ ]:
nest_asyncio.apply()  # Apply the patch to allow nested event loops

async def get_youtube_comments(video_url):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto(video_url)

        # Scroll to load comments
        for _ in range(15):  # Scroll multiple times to load more comments
            await page.evaluate('window.scrollTo(0, document.documentElement.scrollHeight);')
            await page.wait_for_timeout(2000)  # Wait for comments to load

        # Extract comments
        comments = await page.evaluate('''Array.from(document.querySelectorAll('#content-text')).map(el => el.innerText)''')

        await browser.close()
        return comments

# Define the video URL
video_url = 'https://www.youtube.com/watch?v=VQluL1IRDbY'  # Replace with the actual video URL

# Run the scraping function
loop = asyncio.get_event_loop()
comments = loop.run_until_complete(get_youtube_comments(video_url))

# Define the output file path
output_file = '/content/youtube_comments-1.txt'

# Write comments to the file
with open(output_file, 'w') as file:
    for idx, comment in enumerate(comments):
        file.write(f"Comment {idx + 1}: {comment}\n")

print(f"Comments have been written to {output_file}")


Comments have been written to /content/youtube_comments-1.txt


In [ ]:
# Define the file path
input_file = '/content/youtube_comments-1.txt'  # Replace with your file path

# Read the contents of the file
with open(input_file, 'r') as file:
    comments = file.readlines()

# Print the contents of the file
for line in comments:
    print(line.strip())


Comment 1: I've studied your videos and achieved Band 8 in Writing. Well done and thank you
Comment 2: I scored 7.5 overall and 7.5 in writing as well. I took computer based IELTS on the 12th of this month.
Comment 3: Got overall 8 band in just 3 day prep, your videos helped!
Comment 4: this video helps me to analyze those graphs and give me idea for my upcoming ielts exam
Comment 5: I can't thank you enough.The way you made our preparation for IELTS easy is amazing.
Comment 6: Hi Asiya! I can't thank you enough for your amazing content. I got an overall score of 8 with 7.5 in writing in computer based IELTS Academic. Took the test on 24th October. Once again thank you so much. Much love!
Comment 7: Prepared for my academic exam using your videos, managed to score L8.5,R7.5 W7, S8. Thank you
Comment 8: Hi first of all a big thank u from my heart..... I hav passed my Ielts with overall 7.5 with l-8, S 7, Reading -7.5 and W 7....without u nothing would hav been possible... Jst watched ur

In [ ]:


# Read comments from the text file
with open('/content/youtube_comments-1.txt', 'r') as file:
    lines = file.readlines()

# Filter and prepare data for CSV
data = []
for line in lines:
    if line.startswith("Comment"):
        # Extract the comment ID and text
        parts = line.split(':', 1)
        if len(parts) == 2:
            comment_id = parts[0].strip()
            comment_text = parts[1].strip()
            if comment_text:  # Ensure there is text after the colon
                # Append only if it starts with "comment-"
                data.append((comment_id, comment_text))

# Write data to CSV
with open('comments.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['ID', 'Comment'])  # Write header
    writer.writerows(data)  # Write data rows

print("CSV file 'comments.csv' created successfully!")


CSV file 'comments.csv' created successfully!


In [ ]:

# Load CSV file into DataFrame
csv_filename = 'comments.csv'
df=pd.read_csv(csv_filename)
df

,ID,Comment
0,Comment 1,I've studied your videos and achieved Band 8 i...
1,Comment 2,I scored 7.5 overall and 7.5 in writing as wel...
2,Comment 3,"Got overall 8 band in just 3 day prep, your vi..."
3,Comment 4,this video helps me to analyze those graphs an...
4,Comment 5,I can't thank you enough.The way you made our ...
...,...,...
95,Comment 96,Pls can you write the ielts in capital letters...
96,Comment 97,There is nothing that says the two graphs are ...
97,Comment 98,Great
98,Comment 99,11.20 : the predicted rise would be 40%


In [ ]:

# Load the sentiment analysis model
sentiment_pipeline = pipeline('sentiment-analysis')


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [ ]:

# Load CSV file into DataFrame
csv_filename = 'comments.csv'
df = pd.read_csv(csv_filename)

# Function to classify sentiment
def classify_sentiment(text):
    result = sentiment_pipeline(text)
    label = result[0]['label']
    return 'Positive' if label == 'POSITIVE' else 'Negative'

# Apply sentiment analysis to the 'Comment' column
df['Sentiment'] = df['Comment'].apply(classify_sentiment)

# Filter for positive and negative comments
positive_comments = df[df['Sentiment'] == 'Positive']
negative_comments = df[df['Sentiment'] == 'Negative']

# Print positive comments
print("Positive Comments:")
print(positive_comments[['ID', 'Comment']])

# Print negative comments (if needed)
print("\nNegative Comments:")
print(negative_comments[['ID', 'Comment']])


Positive Comments:
             ID                                            Comment
0     Comment 1  I've studied your videos and achieved Band 8 i...
1     Comment 2  I scored 7.5 overall and 7.5 in writing as wel...
2     Comment 3  Got overall 8 band in just 3 day prep, your vi...
3     Comment 4  this video helps me to analyze those graphs an...
4     Comment 5  I can't thank you enough.The way you made our ...
..          ...                                                ...
89   Comment 90  Is it ok to write an overview at the end of th...
91   Comment 92     Please share vocabulary sheet that you've made
97   Comment 98                                              Great
98   Comment 99            11.20 : the predicted rise would be 40%
99  Comment 100                    Hi mam. I'm your new subscriber

[61 rows x 2 columns]

Negative Comments:
            ID                                            Comment
9   Comment 10  The two graphical data represent the alteratio...
14

In [ ]:
positive_comments['Comment']

,Comment
0,I've studied your videos and achieved Band 8 i...
1,I scored 7.5 overall and 7.5 in writing as wel...
2,"Got overall 8 band in just 3 day prep, your vi..."
3,this video helps me to analyze those graphs an...
4,I can't thank you enough.The way you made our ...
...,...
89,Is it ok to write an overview at the end of th...
91,Please share vocabulary sheet that you've made
97,Great
98,11.20 : the predicted rise would be 40%


In [ ]:
# Download required NLTK data
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

# Define POS tags for each part of speech
POS_TAGS = {
    'NN': 'Noun', 'NNS': 'Noun', 'NNP': 'Noun', 'NNPS': 'Noun',  # Nouns
    'JJ': 'Adjective', 'JJR': 'Adjective', 'JJS': 'Adjective',   # Adjectives
    'RB': 'Adverb', 'RBR': 'Adverb', 'RBS': 'Adverb',            # Adverbs
    'VB': 'Verb', 'VBD': 'Verb', 'VBG': 'Verb', 'VBN': 'Verb', 'VBP': 'Verb', 'VBZ': 'Verb'  # Verbs
}

# Auxiliary verbs and adverbs to exclude
auxiliary_verbs = set([
    'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being',
    'have', 'has', 'had', 'having',
    'do', 'does', 'did', 'doing', "'s", "'m", "'re", "n't"
])
specific_adverbs_to_exclude = set([
    'so', "n't", 'just', 'right', 'not', 'here', 'maybe', 'then',
    'often', 'always', 'never', 'rarely',
    'very', 'too', 'quite',
    'everywhere', 'nowhere', 'somewhere', 'anywhere'
])

# Function to filter and count words by POS tag
def filter_words_by_pos(tagged_tokens, pos_tag_prefix):
    return [word.lower() for word, tag in tagged_tokens if tag.startswith(pos_tag_prefix)]

# Function to remove unwanted auxiliary verbs and specific adverbs
def remove_unwanted_words(words):
    return [word for word in words if word not in auxiliary_verbs and word not in specific_adverbs_to_exclude]

# Function to extract text, tokenize, and count word frequencies
def extract_and_analyze_words(json_file, top_n=5):
    with open(json_file, 'r') as file:
        data = json.load(file)

    # Extract text from JSON
    text = ' '.join(segment['text'] for segment in data)

    # Tokenize and POS tagging
    tokens = word_tokenize(text)
    tagged_tokens = pos_tag(tokens)

    # Aggregate filtered words from all POS tags
    all_filtered_words = []
    for pos_tag_key in POS_TAGS:
        filtered_words = filter_words_by_pos(tagged_tokens, pos_tag_key)
        filtered_words = remove_unwanted_words(filtered_words)
        all_filtered_words.extend(filtered_words)

    # Count frequencies of all filtered words
    word_freq = Counter(all_filtered_words)

    # Return the top N most frequent words
    return word_freq.most_common(top_n)

# Call the function and extract all words
subtitle_words = extract_and_analyze_words('/content/subtitle_json.json', top_n=None)

# Convert subtitle_words list of tuples into a set for faster lookup
subtitle_words_set = set(word for word, freq in subtitle_words)

all_common = all_word_freq.most_common()
# Filter the top 5 words from all_common that are present in subtitle.json
top_5_words_in_subtitles = []
for word, freq in all_common:
    if word in subtitle_words_set:
        top_5_words_in_subtitles.append((word, freq))
    if len(top_5_words_in_subtitles) == 5:  # Stop after finding 5 words
        break

# Print the top 5 words
print("Top 5 words found in both all_common and subtitle.json:")
for word, freq in top_5_words_in_subtitles:
    print(f"{word}: {freq}")


Top 5 words found in both all_common and subtitle.json:
words: 22
task: 20
band: 15
charts: 14
data: 13


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
top_frq_word_present_video_and_comment=[]

for word,frq in top_5_words_in_subtitles:
  top_frq_word_present_video_and_comment.append(word)

print(top_frq_word_present_video_and_comment)

['words', 'task', 'band', 'charts', 'data']


In [ ]:
highlighted_subtitle_files = search_words_in_transcription("subtitle_json.json", top_frq_word_present_video_and_comment)
# Extract highlighted sections from the subtitle file
highlighted_sections = extract_highlighted_sections(highlighted_subtitle_files)
# Create highlight video using the extracted sections
create_highlight_video("video.mp4", highlighted_sections)